In [1]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os


In [2]:
dataset_path = "dataset/"  # Path to dataset
train_path = dataset_path  # Your dataset contains humans and non-humans

img_size = (128, 128)  # Resize images for MobileNet
batch_size = 32

# Data Augmentation & Normalization
datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize pixel values
    validation_split=0.2,  # 80% training, 20% validation
)

train_generator = datagen.flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    train_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)


Found 3963 images belonging to 2 classes.
Found 990 images belonging to 2 classes.


In [4]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context


In [5]:
# Load MobileNetV2 (pre-trained on ImageNet), remove top layers
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(128, 128, 3))

# Freeze base model layers (prevent modification during training)
base_model.trainable = False

# Create new model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),  # Flatten features
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # Output layer for binary classification
])

# Compile model
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Model summary
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_128            │ (None, 4, 4, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       163,968 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,422,081 (9.24 MB)

 Trainable params: 164,097 (641.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [6]:
epochs = 10  # Adjust as needed

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=epochs
)

# Save trained model
model.save("mobilenet_human_detection.h5")


Epoch 1/10


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 89ms/step - accuracy: 0.7373 - loss: 0.5309 - val_accuracy: 0.8465 - val_loss: 0.3570
Epoch 2/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - accuracy: 0.8772 - loss: 0.2852 - val_accuracy: 0.8596 - val_loss: 0.3439
Epoch 3/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 11s 90ms/step - accuracy: 0.9213 - loss: 0.1996 - val_accuracy: 0.8465 - val_loss: 0.3545
Epoch 4/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 11s 88ms/step - accuracy: 0.9478 - loss: 0.1546 - val_accuracy: 0.8606 - val_loss: 0.3786
Epoch 5/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 11s 91ms/step - accuracy: 0.9734 - loss: 0.1044 - val_accuracy: 0.8465 - val_loss: 0.3914
Epoch 6/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - accuracy: 0.9916 - loss: 0.0575 - val_accuracy: 0.8475 - val_loss: 0.4265
Epoch 7/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 94ms/step - accuracy: 0.9969 - loss: 0.0338 - val_accuracy: 0.8354 - val_loss: 0.4855
Epoch 8/10
124/124 ━━━━━━━━━━━━━━━━━━━━ 12s 97ms/step - accuracy: 0.9997 - loss: 0.0171 - val_accurac

In [8]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import img_to_array

# Load trained MobileNet model
model = load_model("mobilenet_human_detection.h5")

# Initialize webcam
cap = cv2.VideoCapture(0)
bg_subtractor = cv2.createBackgroundSubtractorMOG2(history=500, varThreshold=50, detectShadows=True)

def classify_object(image):
    """Preprocess the image and classify using MobileNet"""
    image = cv2.resize(image, (128, 128))
    image = img_to_array(image) / 255.0
    image = np.expand_dims(image, axis=0)
    
    prediction = model.predict(image)[0][0]  # Get prediction score

    # If 0 is 'human', adjust thresholding accordingly
    return "Human" if prediction < 0.5 else "Non-Human"

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    fg_mask = bg_subtractor.apply(frame)
    contours, _ = cv2.findContours(fg_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    boxes = []
    for contour in contours:
        if cv2.contourArea(contour) > 500:
            x, y, w, h = cv2.boundingRect(contour)
            boxes.append((x, y, x + w, y + h))

    if boxes:
        x_min = min([box[0] for box in boxes])
        y_min = min([box[1] for box in boxes])
        x_max = max([box[2] for box in boxes])
        y_max = max([box[3] for box in boxes])

        detected_object = frame[y_min:y_max, x_min:x_max]

        if detected_object.size > 0:
            label = classify_object(detected_object)

            color = (0, 255, 0) if label == "Human" else (0, 0, 255)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
            cv2.putText(frame, label, (x_min, y_min - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            if label == "Human":
                print("🚨 Human detected! Triggering Alert!")
                import os
                os.system("say 'Intruder detected'")

    cv2.imshow("Live Motion Detection", frame)

    key = cv2.waitKey(1)
    if key == ord('q') or key == 27:
        print("Manual exit triggered.")
        break

cap.release()
cv2.destroyAllWindows()
print("Exited successfully!")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 291ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
🚨 Human detected! Triggering Alert!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
🚨 Human detected! Triggering Alert!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
🚨 Human detected! Triggering Alert!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
🚨 Human detected! Triggering Alert!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
🚨 Human detected! Triggering Alert!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
🚨 Human detected! Triggering Alert!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


2025-03-25 04:21:28.912 Python[4359:263772] IMKClient Stall detected, *please Report* your user scenario attaching a spindump (or sysdiagnose) that captures the problem - (imkxpc_supportsProperty:reply:) block performed very slowly (1.46 secs).


🚨 Human detected! Triggering Alert!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
🚨 Human detected! Triggering Alert!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
🚨 Human detected! Triggering Alert!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━